## Transportation Problem

In [18]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [19]:
#Create LP model object
mymodel = glp.Solver('Midwest Flour',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [20]:
inf = mymodel.infinity()

         # name: supply capacity, objective coefficient list
sources = {'KC': (170 , [6,8,10 ]),    # Kansas City
           'Om': (250 , [7,11,11 ]),    # Omaha
           'DM': (200 , [4,5,12 ])}    # Des Moines

        # name: demand quantity required
demands = {'Chi': 200, 'StL': 100, 'Cin': 300}    # Chicago, St. Louis, Cincinnati

mymodel.Objective().SetMinimization()

In [21]:
# create all shipment variables and a dictionary of dictionaries to access them
ship = dict()
for s in sources:
    ship[s] = dict()
    (b,coeff_lst) = sources[s]
    for (d,c) in zip(demands,coeff_lst):
        ship[s][d] = mymodel.NumVar(0, inf, s + '_' + d)
        mymodel.Objective().SetCoefficient(ship[s][d] ,c )

In [22]:
print(ship)

{'KC': {'Chi': KC_Chi, 'StL': KC_StL, 'Cin': KC_Cin}, 'Om': {'Chi': Om_Chi, 'StL': Om_StL, 'Cin': Om_Cin}, 'DM': {'Chi': DM_Chi, 'StL': DM_StL, 'Cin': DM_Cin}}


In [23]:
# create demand constraints
for d in demands:
    lb = ub = demands[d]
    constr = mymodel.Constraint(lb ,ub ,d )
    for s in sources:
        constr.SetCoefficient(ship[s][d] , 1)

In [24]:
# create source/supply constraints
for s in sources:
    (b,coeff_lst) = sources[s]
    constr = mymodel.Constraint(-inf ,b ,s )
    for d in demands:
        constr.SetCoefficient(ship[s][d] ,1 )

In [25]:
lpt.print_model(mymodel)

Variables:
KC_Chi, KC_StL, KC_Cin, Om_Chi, Om_StL, Om_Cin, DM_Chi, DM_StL, DM_Cin 

minimize: 6.0*KC_Chi + 8.0*KC_StL + 10.0*KC_Cin + 7.0*Om_Chi + 11.0*Om_StL + 11.0*Om_Cin + 4.0*DM_Chi + 5.0*DM_StL + 12.0*DM_Cin 

Subject To:
Chi: 1.0*KC_Chi + 1.0*Om_Chi + 1.0*DM_Chi = 200.0
StL: 1.0*KC_StL + 1.0*Om_StL + 1.0*DM_StL = 100.0
Cin: 1.0*KC_Cin + 1.0*Om_Cin + 1.0*DM_Cin = 300.0
KC: 1.0*KC_Chi + 1.0*KC_StL + 1.0*KC_Cin <= 170.0
Om: 1.0*Om_Chi + 1.0*Om_StL + 1.0*Om_Cin <= 250.0
DM: 1.0*DM_Chi + 1.0*DM_StL + 1.0*DM_Cin <= 200.0

Bounds:
KC_Chi >= 0.0
KC_StL >= 0.0
KC_Cin >= 0.0
Om_Chi >= 0.0
Om_StL >= 0.0
Om_Cin >= 0.0
DM_Chi >= 0.0
DM_StL >= 0.0
DM_Cin >= 0.0


In [26]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 4730.00
KC_Chi = 0.00
KC_StL = 0.00
KC_Cin = 170.00
Om_Chi = 100.00
Om_StL = 0.00
Om_Cin = 130.00
DM_Chi = 100.00
DM_StL = 100.00
DM_Cin = 0.00


In [27]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
  KC_Chi    0.0    0.0    inf   0.00
  KC_StL    0.0    0.0    inf   1.00
  KC_Cin    0.0  170.0    inf   0.00
  Om_Chi    0.0  100.0    inf   0.00
  Om_StL    0.0    0.0    inf   3.00
  Om_Cin    0.0  130.0    inf   0.00
  DM_Chi    0.0  100.0    inf   0.00
  DM_StL    0.0  100.0    inf   0.00
  DM_Cin    0.0    0.0    inf   4.00


In [28]:
# only print nonzero shipment variables
# print('Variable    LB   Value    UB   Reduced Cost')
# for v in mymodel.variables():
#     if v.solution_value() != 0:
#         print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

In [29]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
       Chi  200.0  200.0  200.0   7.00
       StL  100.0  100.0  100.0   8.00
       Cin  300.0  300.0  300.0  11.00
        KC   -inf  170.0  170.0  -1.00
        Om   -inf  230.0  250.0   0.00
        DM   -inf  200.0  200.0  -3.00
